<a href="https://colab.research.google.com/github/ilovejs/-My-Transport-Protocol/blob/master/Generate_Shakespeare_with_Cloud_TPUs_and_Keras_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Predict Shakespeare with Cloud TPUs and Keras

<table class="tfo-notebook-buttons" align="left">
 <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This example uses [tf.keras](https://www.tensorflow.org/guide/keras) to build a *language model* and train it on a [Google Cloud TPU](https://cloud.google.com/tpu/). This language model predicts the next character of text given the text so far. The trained model can generate new snippets of text that read in a similar style to the text training data.

We'll train the model on the combined works of William Shakespeare, then use it to compose a play in the style of *The Great Bard*:

<blockquote>
Loves that led me no dumbs lack her Berjoy's face with her to-day.  
The spirits roar'd; which shames which within his powers  
	Which tied up remedies lending with occasion,  
A loud and Lancaster, stabb'd in me  
	Upon my sword for ever: 'Agripo'er, his days let me free.  
	Stop it of that word, be so: at Lear,  
	When I did profess the hour-stranger for my life,  
	When I did sink to be cried how for aught;  
	Some beds which seeks chaste senses prove burning;  
But he perforces seen in her eyes so fast;  
And _  
</blockquote>

Note: To enable TPUs on Google Colab, select *Runtime > Change runtime type*, and set *Hardware acceleration* to TPU.

## Download data

Download *The Complete Works of William Shakespeare* as a single text file from [Project Gutenberg](https://www.gutenberg.org/). We'll use snippets from this file as the *training data* for the model. The *target* snippet is offset by one character.

In [0]:
!wget --show-progress --continue -O /content/shakespeare.txt http://www.gutenberg.org/files/100/100-0.txt

--2018-11-20 07:45:03--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852404 (5.6M) [text/plain]
Saving to: ‘/content/shakespeare.txt’

/content/shakespear 100%[===================>]   5.58M  8.40MB/s    in 0.7s    

2018-11-20 07:45:09 (8.40 MB/s) - ‘/content/shakespeare.txt’ saved [5852404/5852404]



### Build the data generator

In [0]:
import numpy as np
import six
import tensorflow as tf
import time
import os

# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

SHAKESPEARE_TXT = '/content/shakespeare.txt'

tf.logging.set_verbosity(tf.logging.INFO)

def transform(txt, pad_to=None):
  # drop any non-ascii characters
  output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  if pad_to is not None:
    # padding
    output = output[:pad_to]
    # which direction ?
    output = np.concatenate([
        # negative
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output


def training_generator(seq_len=100, batch_size=1024):
  """A generator yields (source, target) arrays for training."""
  with tf.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
    txt = f.read()

  tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
  source = transform(txt)
  while True:
    offsets = np.random.randint(0, len(source) - seq_len, batch_size)

    # Our model uses sparse crossentropy loss, but Keras requires labels
    # to have the same rank as the input logits.
    
    # We add an empty final dimension to account for this.
    yield (
        np.stack([source[idx:idx + seq_len] for idx in offsets]),
        np.expand_dims(
            np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
            -1),
    )

# six
six.next(training_generator(seq_len=10, batch_size=1))

INFO:tensorflow:Input text [5834393] ﻿
Project Gutenberg’s The Complete Works of Willi


(array([[101,  32,  98, 101, 101, 110,  32,  97, 115,  32]], dtype=int32),
 array([[[ 32],
         [ 98],
         [101],
         [101],
         [110],
         [ 32],
         [ 97],
         [115],
         [ 32],
         [ 98]]], dtype=int32))

## Build the model

The model is defined as a two-layer, forward-LSTM—with two changes from the `tf.keras` standard LSTM definition:

1. Define the input `shape` of our model which satisfies the [XLA compiler](https://www.tensorflow.org/performance/xla/)'s static shape requirement.
2. Use `tf.train.Optimizer` instead of a standard Keras optimizer (Keras optimizer support is still experimental).

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', 
      shape=(seq_len,),
      batch_size=batch_size, dtype=tf.int32)
  
  # embedding
  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  # time distributed
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  
  model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
  
  model.compile(
      optimizer = tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy']
  )
  
  return model

## Train the model

The `tf.contrib.tpu.keras_to_tpu_model` function converts a `tf.keras` model to an equivalent TPU version. We then use the standard Keras methods to train: `fit`, `predict`, and `evaluate`.

In [0]:
tf.keras.backend.clear_session()

# lstm
training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

# convert
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy = tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    )
)

# training_generator ?
tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=10,
)

tpu_model.save_weights('/tmp/bard.h5', overwrite=True)

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.87.171.170:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 2718395615034514519)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12979318779106883710)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 15282199333139966198)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 599990955127771958)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7217126121979758590)
INFO:tensorflow:*** Available Device: _Device

## Make predictions with the model

Use the trained model to make predictions and generate your own Shakespeare-esque play.
Start the model off with a *seed* sentence, then generate 250 characters from it. We'll make five predictions from the initial seed.

In [0]:
BATCH_SIZE = 5
PREDICT_LEN = 250

# R:
# Keras requires the batch size be specified ahead of time for stateful models.
# 
# We use a sequence length of 1, as we will be feeding in one character at a time and predicting the next character.

prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)

prediction_model.load_weights('/tmp/bard.h5')


# We seed the model with our initial string, copied BATCH_SIZE times
seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)


# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

  
# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(256, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  

for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  generated = ''.join([chr(c) for c in p])
  print(generated)
  print()
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 Sir.

[Exeunt.]

SCENE VII. The same. SCENE I. Tybalts. Clouds opoledy.
E, he may guess.
Now, cousin, and have, with earth, thy kind over-prying.
Let me stay; for brief of Rome, no more of that;
You are all what faith to woo.
Does God bless

PREDICTION 1


 [Musickan] Morrow, Antony,
    ere our heart's right was to Tarquin's charmed?
    At Salisbury, as every made roare,
    Almost in the earth, but for a burden besport.
  SECOND GENTLEMAN. It is some itch within my steward. If this mark
    Is 

PREDICTION 2


 If heaven, good Masker, hold their distracted villains,
    And from hence oer the world had won.
    And then this frowning water employed,
    Disbellow combast summer, if thou dost dislaive, all
    month's clospity, that words up in Eastchea

PREDICTION 3


 Loeve
    all; and if we shall, and tooth as swiftly honourd.
  WARWICK. Brother, there, good Master.
  LONGAVILLE. It is a disauture, you know, if not on, that degenounce is for old
    yet of peace, 